## Import

In [17]:
import os
import matplotlib.pyplot as plt
import datetime
import numpy as np
import pandas as pd
import string
from itertools import cycle;
import seaborn as sns
import re
import random
%matplotlib inline

In [16]:
# # Gensim
# import gensim
# from gensim.utils import simple_preprocess
# from gensim.parsing.preprocessing import STOPWORDS
# from gensim import corpora, models
# import pyLDAvis
# import pyLDAvis.gensim
# import plotly.graph_objs as go
# from sklearn.cluster import KMeans;
# from sklearn.neighbors import KDTree;
# from gensim.models import Word2Vec
# import plotly.offline as py

# Preprocessors
# import spacy
# from gensim.models import CoherenceModel

# NLTK and WordCloud
# from wordcloud import WordCloud
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")
from nltk.stem import WordNetLemmatizer, SnowballStemmer, PorterStemmer
from nltk.stem.porter import *
from sklearn.feature_extraction.text import CountVectorizer
import nltk

In [27]:
import pandas as pd
import re

In [26]:
actual = pd.read_csv("January_Data")
# data = pd.read_csv("DIET_NUTRITION_IMMUNITY.csv")

In [28]:
actual = actual[:100]
actual.to_csv('DATA_TEST.csv')

In [68]:
base = list(data['0'])

In [3]:
API_KEY = '1m6IsbpYH3qZuN0YhotEeYv1fF8j8FF9aq_YX-I9qwmo'
URL = 'https://api.eu-gb.natural-language-understanding.watson.cloud.ibm.com/instances/1ace0f79-7bef-47ea-a1a7-87ea93435f2d'

In [19]:
# actual["Data"] = actual["Tweet"]
del actual["Location"], actual["Retweets"], actual["Timestamp"]
actual = actual[:100]

In [21]:
actual.head()

,Tweet
0,RT @AnneKPIX: @CDC has activated its emergency...
1,RT @OurWarOnCancer: Where is our FEDERAL vacci...
2,RT @nytimes: Breaking News: The first U.S. cas...
3,RT @ScottAnthonyUSA: ⚠️ IT SHOULD BE NOTED tha...
4,RT @CDCgov: PHAP gives future #publichealth pr...


In [22]:
actual["Data"] = actual["Tweet"]

In [23]:
del actual["Tweet"]

# Word2vec

In [ ]:
# actual.head()

In [60]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def stem(text):
    return stemmer.stem(text)

def tweeter(text):
    text = text.lower()
    if (text.startswith('rt')):
        text = text[2:]
    tw = text
    tw = re.sub(r'\n',' ', tw) # Remove line breaks
    tw = re.sub('\s+', ' ', tw).strip()# Remove leading, trailing, and extra spaces
    w = re.sub(r'https?://\S+', '', tw) # Remove links
    tw = re.sub(r'http?://\S+', '', tw)  #Remove links
    tw = re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tw)
    return tw
    
def preprocess(text):
    result = []
    text = re.sub(r"http\S+", "", text)
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and (len(token) > 3) and (token != 'http'):
            result.append(token)
    return ' '.join(result) 

<input>:13: DeprecationWarning:

invalid escape sequence \s

<input>:16: DeprecationWarning:

invalid escape sequence \w

<input>:13: DeprecationWarning:

invalid escape sequence \s

<input>:16: DeprecationWarning:

invalid escape sequence \w

<input>:13: DeprecationWarning:

invalid escape sequence \s

<input>:16: DeprecationWarning:

invalid escape sequence \w

<ipython-input-60-8f84cb9ac8c5>:13: DeprecationWarning:

invalid escape sequence \s

<ipython-input-60-8f84cb9ac8c5>:16: DeprecationWarning:

invalid escape sequence \w



In [ ]:
actual["Data"] = actual.Data.apply(tweeter)
docs = actual['Data'].map(preprocess)
docs[:5]

In [ ]:
# jdoc = []
# for i in docs:
#     jdoc.append(i)
    
# model = Word2Vec(jdoc, min_count=1, workers=2)
# words = list(model.wv.vocab)
# print(words)

# Clustering

In [ ]:
X = model[model.wv.vocab]

In [ ]:
def clustering_on_wordvecs(word_vectors, num_clusters):
    # Initalize a k-means object and use it to extract centroids
    kmeans_clustering = KMeans(n_clusters = num_clusters, init='k-means++');
    idx = kmeans_clustering.fit_predict(word_vectors);
    
    return kmeans_clustering.cluster_centers_, idx;

In [ ]:
centers, clusters = clustering_on_wordvecs(X, 20)
centroid_map = dict(zip(model.wv.index2word, clusters))

In [ ]:
def get_top_words(index2word, k, centers, wordvecs):
    tree = KDTree(wordvecs);
#Closest points for each Cluster center is used to query the closest 20 points to it.
    closest_points = [tree.query(np.reshape(x, (1, -1)), k=k) for x in centers];
    closest_words_idxs = [x[1] for x in closest_points];
#Word Index is queried for each position in the above array, and added to a Dictionary.
    closest_words = {};
    for i in range(0, len(closest_words_idxs)):
        closest_words['Cluster #' + str(i)] = [index2word[j] for j in closest_words_idxs[i][0]]
#A DataFrame is generated from the dictionary.
    df = pd.DataFrame(closest_words);
    df.index = df.index+1
    return df

In [ ]:
top_words = get_top_words(model.wv.index2word, 2000, centers, X);

In [ ]:
top_words.head()

In [ ]:
def display_cloud(cluster_num, cmap):
    wc = WordCloud(background_color="black", max_words=100, max_font_size=80, colormap=cmap);
    wordcloud = wc.generate(' '.join([word for word in top_words['Cluster #' + str(cluster_num)]]))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.savefig('cluster_' + str(cluster_num), bbox_inches='tight')

In [ ]:
cmaps = cycle([
            'flag', 'prism', 'ocean', 'gist_earth', 'terrain', 'gist_stern',
            'gnuplot', 'gnuplot2', 'CMRmap', 'cubehelix', 'brg', 'hsv',
            'gist_rainbow', 'rainbow', 'jet', 'nipy_spectral', 'gist_ncar'])
for i in range(20):
    col = next(cmaps);
    display_cloud(i, col)

## ----------------------------------------------------------------------------------------------

## ----------------------------------------------------------------------------------------------

## Attempting Spacy Vectoriser

In [77]:
# base

In [47]:
import spacy
spacy.prefer_gpu()
import en_core_web_sm
nlp = en_core_web_sm.load()
from spacy import displacy
from collections import Counter
from spacy.matcher import Matcher

In [73]:
def simcalc(df, base):
    ans = {}
    for tw in df["Tweet"]:
        total = 0
        for b in base:
            t1 = nlp(tw)
            t2 = nlp(b)
            total += t1.similarity(t2)
        total = total/len(b)
        ans[tw] = total
    return ans

In [61]:
actual["Tweet"] = actual.Tweet.apply(tweeter)
actual["Tweet"] = actual.Tweet.apply(preprocess)

In [74]:
ans = simcalc(actual, base)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/runpy.py:193: ModelsWarning:

[W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/runpy.py:193: ModelsWarning:

[W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vector

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/runpy.py:193: ModelsWarning:

[W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/runpy.py:193: ModelsWarning:

[W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vector

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/runpy.py:193: ModelsWarning:

[W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/runpy.py:193: ModelsWarning:

[W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vector

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/runpy.py:193: ModelsWarning:

[W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/runpy.py:193: ModelsWarning:

[W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vector

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/runpy.py:193: ModelsWarning:

[W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/runpy.py:193: ModelsWarning:

[W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vector

...

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/runpy.py:193: ModelsWarning:

[W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/runpy.py:193: ModelsWarning:

[W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vector

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/runpy.py:193: ModelsWarning:

[W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/runpy.py:193: ModelsWarning:

[W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vector

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/runpy.py:193: ModelsWarning:

[W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/runpy.py:193: ModelsWarning:

[W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vector

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/runpy.py:193: ModelsWarning:

[W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/runpy.py:193: ModelsWarning:

[W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vector

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/runpy.py:193: ModelsWarning:

[W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/runpy.py:193: ModelsWarning:

[W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vector

In [75]:
ans

{'RT @AnneKPIX: @CDC has activated its emergency operations center. \nThey expect more US cases.\n#coronavirus': 5.724234697929972,
 'RT @OurWarOnCancer: Where is our FEDERAL vaccination education campaign for HPV?! Where is our FEDERAL #lungcancer screening program?! (and…': 4.3078833081524515,
 'RT @nytimes: Breaking News: The first U.S. case of the Wuhan coronavirus has been confirmed in Washington State, federal officials said. Th…': 5.1627171569269175,
 'RT @ScottAnthonyUSA: ⚠️ IT SHOULD BE NOTED that the CDC initially had an embargo placed on CDC testimony.  The TRUMP ADMINISTRATION LIFTED…': 5.005555806902647,
 'RT @CDCgov: PHAP gives future #publichealth professionals critical frontline experience. Apply now! https://t.co/8kzRD1IADl https://t.co/ha…': 5.458386977755804,
 'RT @PMBreakingNews: Breaking: The CDC is expected to announce the first case of the new strain of coronavirus, or nCov, in Washington state…': 4.952659466193991,
 'RT @Reuters: The U.S. Centers for Disease Con

## ----------------------------------------------------------------------------------------------

## ----------------------------------------------------------------------------------------------

# Usage

In [10]:
# from extract import extract
from extract import extract
from analyse import analyse
from visualise import visualise

In [24]:
inst = extract(actual=actual, sample_size=10)

In [25]:
final = inst.extract_features(API_KEY=API_KEY, URL=URL, chunksize=200)

AttributeError: 'Series' object has no attribute 'lower'

In [ ]:
vis = visualise(data = final)

In [ ]:
vis.textvis()

In [ ]:
an = analyse(data=final)

In [ ]:
cats = an.catanalyse(category='health and fitness')
keys = an.keyanalyse()

In [ ]:
cats

In [ ]:
keys